# Exploratory Data Analysis

In [216]:
# import libraries
import os
import sys
import urllib.request
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Naver News
### 1-1. Naver API

In [ ]:
client_id = "sPoIaxsqZqCViA3jWiOG"
client_secret = "zb_x9LGf4b"

In [211]:
encText = urllib.parse.quote("전기차 보조금") # search term
display = 15
sort = "date"

url = "https://openapi.naver.com/v1/search/news?query=" + encText \
    + "&display=" + str(display) \
    + "&sort=" + sort

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    json_str = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)
    
json_object = json.loads(json_str) # to json
df = pd.DataFrame(json_object['items']) # to dataframe
df

,title,originallink,link,description,pubDate
0,"中 BYD, 승용 <b>전기차</b> 韓 판매 가시권?…돌핀 등 6개 상표 출원",https://www.news1.kr/articles/4859420,https://n.news.naver.com/mnews/article/421/000...,"업계 관계자는 &quot;전세계가 보호무역주의로 움직이고 있는데, 중국산 <b>전기...","Fri, 11 Nov 2022 10:13:00 +0900"
1,"인플레이션 감축법 대응…韓, 국내 투자 지원 확대 필요",https://www.smartfn.co.kr/article/view/sfn2022...,https://www.smartfn.co.kr/article/view/sfn2022...,"반도체, <b>전기차</b> 등 국내 대기업의 대규모 해외투자가 예상돼 투자액이 해...","Fri, 11 Nov 2022 10:10:00 +0900"
2,한경연 &quot;美 IRA 대비해 광물 수입 다각화·리쇼어링 범위 확대해야&quot;,http://www.inews24.com/view/1537924,https://n.news.naver.com/mnews/article/031/000...,기업에 <b>전기차</b> 신차에 대당 최대 7천500 달러의 세액공제를 적용한다....,"Fri, 11 Nov 2022 09:50:00 +0900"
3,도심형 거점 <b>전기차</b>충전소 구축 나선다,http://www.e2news.com/news/articleView.html?id...,http://www.e2news.com/news/articleView.html?id...,이인근 서울시 기후환경기획관은 &quot;다른 공공기관과 협력을 통해 <b>전기차<...,"Fri, 11 Nov 2022 09:46:00 +0900"
4,[다시보기] 11월 10일 (목) 오후 사이언스 투데이,https://science.ytn.co.kr/program/view.php?mcd...,https://science.ytn.co.kr/program/view.php?mcd...,정부 청사진 공개 ■ &apos;적자&apos; 수소충전소 93곳에 평균 3천여만 ...,"Fri, 11 Nov 2022 09:42:00 +0900"
5,"볼린저의 새로운 배터리 파트너, <b>전기차</b> 혁명 가능할까?",https://www.autodaily.co.kr/news/articleView.h...,https://www.autodaily.co.kr/news/articleView.h...,이번 파트너십은 곤경에 처한 미국 <b>전기차</b> 제조사에게 있어 두 번째 큰 ...,"Fri, 11 Nov 2022 09:40:00 +0900"
6,IRA로 생산시설 미국 쏠림 우려…연구비·세제혜택 늘려야,https://www.ajunews.com/view/20221111062630932,https://www.ajunews.com/view/20221111062630932,"북미 내 최종 조립, 배터리 핵심 광물 조달국 비율 충족 등 요건을 맞춘 <b>전기...","Fri, 11 Nov 2022 09:38:00 +0900"
7,"울산시, 내년도 예산 전년 대비 4.4% 증액 및 재정건전성 전환",https://www.viva100.com/main/view.php?key=2022...,https://www.viva100.com/main/view.php?key=2022...,"4%) 증가한 금액으로, 국고<b>보조금</b>, 보통교부세 등 국비 증가와 지방세...","Fri, 11 Nov 2022 09:32:00 +0900"
8,"미국 자동차 업계, 가상 검증(Virtual Validation) 도입으로 보는 기...",http://dream.kotra.or.kr/kotranews/cms/news/ac...,http://dream.kotra.or.kr/kotranews/cms/news/ac...,현재 대부분의 자동차 제조 기업들이 <b>전기차</b>를 출시했거나 출시를 앞두고 ...,"Fri, 11 Nov 2022 09:28:00 +0900"
9,美 인플레 감축법 발효… 반도체 특별법(K칩스법) 빠른 통과 필요해,http://www.gvalley.co.kr/news/articleView.html...,http://www.gvalley.co.kr/news/articleView.html...,"반도체, <b>전기차</b> 등 국내 대기업의 대규모 해외투자가 예상되어 투자액이 ...","Fri, 11 Nov 2022 09:28:00 +0900"


### 1-2. Naver News

* Chosun
* Donga
* Hani
* Joongang
* Khan

In [217]:
!pip install dash
import plotly.express as px
import plotly.graph_objects as go

In [218]:
news = pd.read_csv("/Users/2hyunie/Documents/qmss_thesis/Datasets/raw/articles_ev.csv")
news.head()

,date,khan,hani,chosun,donga,joongang
0,2018/01,28,27,0,63,0
1,2018/02,30,20,0,60,0
2,2018/03,16,24,0,57,0
3,2018/04,23,20,0,49,0
4,2018/05,30,18,0,35,0


In [219]:
news['date'] = pd.to_datetime(news['date'])
news.dtypes

date        datetime64[ns]
khan                 int64
hani                 int64
chosun               int64
donga                int64
joongang             int64
dtype: object

In [220]:
news['mean'] = news.mean(numeric_only=True, axis=1)
news.head()

,date,khan,hani,chosun,donga,joongang,mean
0,2018-01-01,28,27,0,63,0,23.6
1,2018-02-01,30,20,0,60,0,22.0
2,2018-03-01,16,24,0,57,0,19.4
3,2018-04-01,23,20,0,49,0,18.4
4,2018-05-01,30,18,0,35,0,16.6


In [221]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = news['date'], y = news['chosun'],
             name = 'The Chosun Ilbo', mode = 'lines'))
fig.add_trace(go.Scatter(x = news['date'], y = news['donga'],
             name = 'The Dong-a Ilbo', mode = 'lines'))
fig.add_trace(go.Scatter(x = news['date'], y = news['hani'],
             name = 'The Hankyoreh', mode = 'lines'))
fig.add_trace(go.Scatter(x = news['date'], y = news['joongang'],
             name = 'JoongAng Ilbo', mode = 'lines'))
fig.add_trace(go.Scatter(x = news['date'], y = news['khan'],
             name = 'Kyunghyang Shinmun', mode = 'lines'))

fig.update_layout(title = 'Count of Articles on "Electric Vehicles"',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white')

fig.show()

In [214]:
fig = px.bar(news, x = 'date',
             y = ['chosun','donga','hani','joongang','khan'])
fig.update_layout(title = 'Count of Articles on "Electric Vehicles"',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white')
fig.show()

## 2. Google trends

In [135]:
!pip install pytrends

In [136]:
from pytrends.request import TrendReq

In [215]:
pytrends = TrendReq(hl='ko', tz=540)

kw_list = ["전기차", "친환경"]

pytrends.build_payload(kw_list, cat=0, timeframe="2020-02-05 2022-11-01", geo="KR")
google = pytrends.interest_over_time()
google = google.reset_index()

fig = px.line(google, x="date", y=kw_list,
              title="Google Trends in South Korea",
             labels={
                 "variable" : "Search Term", "date" : "Time", "value" : "Popularity"
             }, template = 'simple_white')

fig.show()

In [ ]:
google = google.drop('isPartial', axis=1)
google.head()

In [184]:
pytrends = TrendReq(hl='ko', tz=540)

kw_list = ["전기차"]

pytrends.build_payload(kw_list, cat=0, timeframe="2018-01-01 2019-12-31", geo="KR")
trends = pytrends.related_queries()
top1 = trends["전기차"]["top"]

## 3. Covid-19 Data

link: https://115.84.165.39/dataList/OA-20461/S/1/datasetView.do

In [148]:
# retrieving data

covid = pd.read_csv("/Users/2hyunie/Documents/qmss_thesis/Datasets/raw/covid_data.csv")

In [149]:
# selecting necessary columns

covid = covid.iloc[:, [0,1,2,6,11,12,15]]

In [150]:
# changing column names

covid.columns = ['date', 'seoul_pat', 'seoul_new', 'seoul_death', 'nat_pat', 'nat_new', 'nat_death']

In [151]:
covid.dtypes

date            object
seoul_pat        int64
seoul_new        int64
seoul_death      int64
nat_pat        float64
nat_new          int64
nat_death        int64
dtype: object

In [152]:
covid['date'] = pd.to_datetime(covid['date'])

In [153]:
covid.dtypes

date           datetime64[ns]
seoul_pat               int64
seoul_new               int64
seoul_death             int64
nat_pat               float64
nat_new                 int64
nat_death               int64
dtype: object

In [154]:
covid.head()

,date,seoul_pat,seoul_new,seoul_death,nat_pat,nat_new,nat_death
0,2022-11-01,5003593,13017,5608,25615667.0,58379,29209
1,2022-10-31,4990582,3099,5603,25557309.0,18510,29176
2,2022-10-30,4987483,6708,5598,25538799.0,34511,29158
3,2022-10-29,4980775,7578,5597,25504288.0,37321,29131
4,2022-10-28,4973197,7480,5594,25466992.0,35924,29100


In [155]:
x = covid['date']
y = covid['nat_new']

fig = px.line(covid, x, y, title="Daily Covid-19 Cases",
             labels={
                 "date" : "Date", "nat_new" : "New Patients"
             }, template = 'simple_white')
fig.show()

## 4. EV Sales Data

In [156]:
hyundai = pd.read_csv('/Users/2hyunie/Documents/qmss_thesis/Datasets/raw/hyundai.csv')
hyundai.head()

,date,avante_hev_d,ioniq_hev_d,ioniq_phev_d,ioniq_ev_d,sonata_hev_d,sonata_phev_d,sonata_dn8_hev_d,grandeur_hev_d,g80_ev_d,...,sonata_dn8_hev_e,grandeur_hev_e,g80_ev_e,kona_hev_e,kona_ev_e,tuscon_hev_e,tuscon_phev_e,santa_hev_e,santa_phev_e,gv70_ev_e
0,2018-01,NaN,207.0,5.0,1086.0,289.0,1.0,NaN,1939,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02,NaN,322.0,11.0,949.0,294.0,4.0,NaN,1625,NaN,...,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
2,2018-03,NaN,201.0,10.0,886.0,304.0,4.0,NaN,1945,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,2018-04,NaN,353.0,14.0,485.0,333.0,1.0,NaN,1728,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,2018-05,NaN,259.0,7.0,548.0,347.0,4.0,NaN,2521,NaN,...,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN


In [157]:
hyundai = hyundai.fillna(0)
hyundai['date'] = pd.to_datetime(hyundai['date'])
hyundai.head()

,date,avante_hev_d,ioniq_hev_d,ioniq_phev_d,ioniq_ev_d,sonata_hev_d,sonata_phev_d,sonata_dn8_hev_d,grandeur_hev_d,g80_ev_d,...,sonata_dn8_hev_e,grandeur_hev_e,g80_ev_e,kona_hev_e,kona_ev_e,tuscon_hev_e,tuscon_phev_e,santa_hev_e,santa_phev_e,gv70_ev_e
0,2018-01-01,0.0,207.0,5.0,1086.0,289.0,1.0,0.0,1939,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-02-01,0.0,322.0,11.0,949.0,294.0,4.0,0.0,1625,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
2,2018-03-01,0.0,201.0,10.0,886.0,304.0,4.0,0.0,1945,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,2018-04-01,0.0,353.0,14.0,485.0,333.0,1.0,0.0,1728,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,2018-05-01,0.0,259.0,7.0,548.0,347.0,4.0,0.0,2521,0.0,...,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0


In [158]:
hyundai['mean'] = hyundai.mean(numeric_only=True, axis=1)

In [159]:
for col in hyundai.columns:
    print(col)

date
avante_hev_d
ioniq_hev_d
ioniq_phev_d
ioniq_ev_d
sonata_hev_d
sonata_phev_d
sonata_dn8_hev_d
grandeur_hev_d
g80_ev_d
kona_hev_d
kona_ev_d
tuscon_hev_d
santa_hev_d
gv70_ev_d
avante_hev_e
ioniq_hev_e
ioniq_phev_e
ioniq_ev_e
sonata_hev_e
sonata_phev_e
sonata_dn8_hev_e
grandeur_hev_e
g80_ev_e
kona_hev_e
kona_ev_e
tuscon_hev_e
tuscon_phev_e
santa_hev_e
santa_phev_e
gv70_ev_e
mean


In [160]:
fig = px.bar(hyundai, x = 'date',
             y = ['ioniq_ev_d','g80_ev_d','kona_ev_d','gv70_ev_d'])
fig.update_layout(title = 'Count of domestic Hyundai EV sales',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white',
                 showlegend=False)
fig.show()

In [161]:
fig = px.bar(hyundai, x = 'date',
             y = ['avante_hev_d','ioniq_hev_d','sonata_hev_d','sonata_dn8_hev_d','grandeur_hev_d',
                  'kona_hev_d','tuscon_hev_d','santa_hev_d'])
fig.update_layout(title = 'Count of domestic Hyundai HEV sales',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white',
                 showlegend=False)
fig.show()

In [162]:
fig = px.bar(hyundai, x = 'date',
             y = ['ioniq_ev_e','g80_ev_e','kona_ev_e','gv70_ev_e'])
fig.update_layout(title = 'Count of Hyundai EV export sales',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white',
                 showlegend=False)
fig.show()

In [163]:
fig = px.bar(hyundai, x = 'date',
             y = ['avante_hev_e','ioniq_hev_e','sonata_hev_e','sonata_dn8_hev_e','grandeur_hev_e',
                 'kona_hev_e','tuscon_hev_e','santa_hev_e'])
fig.update_layout(title = 'Count of Hyundai HEV export sales',
                 xaxis_title = 'Date',
                 yaxis_title = 'Count',
                 template = 'simple_white',
                 showlegend=False)
fig.show()